# Building a Controller

The following documents the development of a new controller.
In this case we are going to implement an arbitrary controllable storage unit. This
may be a battery, an electrically powered car or some sort of reservoir storage.

## Controller init
First we start by creating a new file *control/storage_control.py*, containing our new class.

In [34]:
    import control

    class Storage(control.basic_controller.Controller):
        """
            Example class of a Storage-Controller. Models an abstract energy storage.
        """
        def __init__(self, net, gid, data_source=None, p_profile=None, in_service=True,
                     recycle=False, order=0, level=0, **kwargs):
            super().__init__(net, in_service=in_service, recycle=recycle, order=order, level=level,
                        initial_powerflow = True, **kwargs)
            # read generator attributes from net
            self.gid = gid #index of the controlled storage
            self.bus = net.storage.at[gid, "bus"]
            self.p_mw = net.storage.at[gid, "p_mw"]
            self.q_mvar = net.storage.at[gid, "q_mvar"]
            self.sn_mva = net.storage.at[gid, "sn_mva"]
            self.name = net.storage.at[gid, "name"]
            self.gen_type = net.storage.at[gid, "type"]
            self.in_service = net.storage.at[gid, "in_service"]
            self.applied = False

            #specific attributes
            self.max_e_mwh = net.storage.at[gid, "max_e_mwh"]
            self.soc_percent = net.storage.at[gid, "soc_percent"]
            
            # profile attributes
            self.data_source = data_source
            self.p_profile = p_profile
            self.last_time_step = None

Import and inherit from the parent class `Controller` and override methods you would like to use. Next we write the actual code for the methods. We choose to represent the storage-unit as a storage
element in pandapower. We start with a function calculating the amout of stored energy:

In [25]:
    def get_stored_ernergy(self):
        # do some "complex" calculations
        return self.max_e_mwh * self.soc_percent / 100

Also a first step we want our controller to be able to write its P and Q and state of charge values back to the
data structure net.
   

In [26]:
def write_to_net(self):
        # write p, q to bus within the net
        self.net.storage.at[self.gid, "p_mw"] = self.p_mw
        self.net.storage.at[self.gid, "q_mvar"] = self.q_mvar
        self.net.storage.at[self.gid, "soc_percent"]= self.soc_percent

Also remember that 'is_converged()' returns the boolean value of convergence:

In [27]:
    def is_converged(self):
        # check if controller already was applied
        return self.applied

In case the controller is not yet converged, the control step is executed. In the example it simply
adopts a new value according to the previously calculated target and writes back to the net.

In [28]:
    def control_step(self):
        # write p, q, soc to storage within the net
        self.write_to_net()
        self.applied = True


In a time-series simulation the battery should read new power values from a profile and keep track
of its state of charge as depicted below.

In [29]:
    def time_step(self, time):
        # keep track of the soc (assuming time is given in seconds)
        if self.last_time_step is not None:
            self.soc_percent += self.max_e_mwh / (self.p_mw * (self.current_time_step-self.last_time_step) / 3600)
        self.last_time_step = time

        # read new values from a profile
        if self.data_source:
            if self.p_profile:
                self.p_kw = self.data_source.get_time_step_value(time_step=time,
                                                                profile_name=self.p_profile)
                
        self.applied = False # reset applied variable

We are now ready to create objects of our newly implemented class and simulate with it!

In [36]:
# Importing necessary packages
import pandapower as pp
import control
import pandas as pd
import timeseries as ts

from pandapower.networks import mv_oberrhein
# loading the network with the usecase 'generation'
net = mv_oberrhein()
pp.runpp(net)

store_el = pp.create_storage(net, 30, p_mw = .1, q_mvar = 0, max_e_mwh = 1)
framedata = pd.DataFrame([0.1,.05,0.1,.005])
datasource = ts.DFData(framedata)

ow = ts.OutputWriter(net)

ow.log_variable("res_storage", "p_mw")
ow.log_variable("storage", "soc_percent")

ctrl = Storage(net=net, gid=store_el, data_source=datasource,p_profile=0 )

ts.run_timeseries(net, time_steps=range(0, 4), output_writer=ow)

print(ow.output)

hp.pandapower.control.util.auxiliary - INFO: Creating controller 0 of type <class '__main__.Storage'> 
hp.pandapower.control.basic_controller - WARNING: Method is_converged() has not been overwritten (and will always return True)!
hp.pandapower.control.basic_controller - WARNING: Method is_converged() has not been overwritten (and will always return True)!
hp.pandapower.control.basic_controller - WARNING: Method is_converged() has not been overwritten (and will always return True)!
hp.pandapower.control.basic_controller - WARNING: Method is_converged() has not been overwritten (and will always return True)!


Progress: |██████████████████████████████████████████████████| 100.0% Complete

{'Parameters':    time_step  controller_unstable  powerflow_failed
0          0                False             False
1          1                False             False
2          2                False             False
3          3                False             False, 'res_storage.p_mw':      0
0  0.1
1  0.1
2  0.1
3  0.1, 'storage.soc_percent':     0
0 NaN
1 NaN
2 NaN
3 NaN}
